In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 75
plt.rcParams['font.size'] = 12
sns.set_theme(style="whitegrid")
sns.set_theme(color_codes=True)
sns.set_context("poster")

import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import statsmodels.graphics.api as smg
from statsmodels.miscmodels.ordinal_model import OrderedModel

import rpy2.rinterface
import rpy2.robjects as robjects
%load_ext rpy2.ipython

In [21]:
%%R -o uk
load('seminar2 (2).RData')

Use data frame ‘uk’ from ‘seminar2 (2).RData’.  

The variables are:
- **polintr** -  how interested in politics; 1 - Not at all interested, 4 - Very interested;
- **age** – age of the respondent
- **sex** – gender of the respondent
- **edu** – age when completed education
- **income1** – income group

Fulfil the following tasks:
1. Predict **interest in politics** by *age*, *gender*, *education* and *income group*.
2. Interpret the coefficients.
3. Tell if the model fits data well.
4. Visualize the link between interest in politics and age when respondent completed education.

In [22]:
uk.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2422 entries, 1 to 2422
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   polintr  2420 non-null   category
 1   vote     1559 non-null   category
 2   age      2413 non-null   float64 
 3   sex      2422 non-null   category
 4   edu      2395 non-null   float64 
 5   income   1906 non-null   category
 6   incdiff  2344 non-null   category
 7   health   2416 non-null   category
 8   prvote   1559 non-null   category
 9   income1  1906 non-null   float64 
dtypes: category(7), float64(3)
memory usage: 93.9+ KB


In [23]:
uk = uk[['polintr', 'age', 'sex', 'edu', 'income1']]
uk = uk.dropna()

In [24]:
uk.polintr = uk.polintr.values.set_categories(['Not at all interested', 'Hardly interested', 'Quite interested', 'Very interested'], ordered=True)

## 1. Predict **interest in politics** by *age*, *gender*, *education* and *income group*.

In [30]:
uk.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1892 entries, 1 to 2422
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   polintr  1892 non-null   category
 1   age      1892 non-null   float64 
 2   sex      1892 non-null   category
 3   edu      1892 non-null   float64 
 4   income1  1892 non-null   float64 
dtypes: category(2), float64(3)
memory usage: 127.7+ KB


In [31]:
# probit ordinal regression
mod_prob = OrderedModel(np.asarray(uk['polintr']),
                        uk[['age', 'sex', 'edu', 'income1']],
                        distr='probit')

res_prob = mod_prob.fit(method='bfgs')
res_prob.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

## 2. Interpret the coefficients.

## 3. Tell if the model fits data well.

## 4. Visualize the link between interest in politics and age when respondent completed education.

In [ ]:
# Constructing a model
model = smf.logit(formula='concert1 ~ age + gender + commun + bill + edu', data=culture).fit()
print(model.summary())

## Results interpretation:

In [ ]:
model.pvalues < 0.05

**All coefficients are significant!**

In [ ]:
# Odds Ratio
round(np.exp(model.params), 3)

- **gender[T.Female]**: changing of gender to **Female** multiplies odds of concert attendance by **1.075** times


- **commun[T.Small/middle town]**: changing of community type to **Small/middle town** multiplies odds of concert attendance by **1.222** times
- **commun[T.Large town]**: changing of community type to **Large town** multiplies odds of concert attendance by **1.281** times


- **bill[T.no]**: having difficulties with paying bills (in comparison with thhe absence of such difficulties) multiplies odds of concert attendance by **1.821** times


- **age**: increase in one year of age multiplies odds of concert attendance by **0.982** times
- **edu**: increase in one year of edu multiplies odds of concert attendance by **1.129** times

In [ ]:
# Average Marginal Effects
print(model.get_margeff().summary())

- **gender[T.Female]**: changing of gender to Female **increases** the probability of concert attendance by 1.49%


- **commun[T.Small/middle town]**: ichanging of community type to Small/middle town **increases** the probability of concert attendance by 4.12%
- **commun[T.Large town]**: changing of community type to Large town **increases** the probability of concert attendance by 5.1%


- **bill[T.no]**: having difficulties with paying bills (in comparison with thhe absence of such difficulties) **increases** the probability of concert attendance by 12.33%


- **age**: increase in one year of age **decreases** the probability of concert attendance by 0.37%
- **edu**: increase in one year of edu **increases** the probability of concert attendance by 2.49%

### Overall

- **gender** doesn't have strong effect

- the larger **community type** the bigger probability to attend concert (*I suppose just because of bigger amount of concerts in larger towns*)

- **bill** (or having difficulties of paying bills) have quite strong *at least among other variables* effect on concert attendance: **for having problems with bills probability increases by 12%**

- **age** seems to have no strong effect on probability of concert attendance

- and more **educated** ones will have bigger chances of concert attendance

In [ ]:
# Check for quadratic relationships
# Constructing a model
model0 = smf.logit(formula='concert1 ~ np.power(age, 2) + gender + commun + bill + edu', data=culture).fit()
print(model0.summary())

## Note on quadratic relationship of age & concert attendance

Looks like there *is no any significant and/or strong relationships* between those two variables - neither confidence intervals are reliable (-0 is not goot CI at all) or the coefficient value almost at 0 too.

In [ ]:
sm.graphics.plot_partregress(endog='concert1', exog_i='age', exog_others=['gender', 'commun', 'bill', 'edu'], data=culture, obs_labels=False);

## Plot interpretation
- As can be seen **bigger group** of observations concentrated in lower part - larger amount of people not attend concerts.
- And with more years of age lower group are **slowly growing in size** - more people with ages tend to not attend concerts.

From those two observations can be derived simple assumption (which is already confirmed by the model) that **with more years of age probability of attending on concerts decreasing very slowly.**

In [ ]:
# Pseudo R^2
print('Pseudo R^2:', model.summary().tables[0][3][3])

In [ ]:
# PCP
predicted = [np.where(i == i.max())[0][0] for i in model.predict()]
print('Percentage of Correct Predictions (PCP): ', (round(sum(predicted == culture.concert1)/len(culture)*100, 1)), '%', sep='')

## Model fit:
- **Pseudo-R squared** is below 0.1 so model can be considered as poor-mediocre fit
- **PCP or Accuracy** equals to 63.8% which is anyway better that 50/50 guessing!

# Task 2: **values**

Select only respondents from Finland.
1. Specify a model to predict the value of achievement by other variables.
2. Interpret the results
3. Check if there is an interaction effect between respondent’s age and respondent’s gender.
4. Visualize the interaction effect, interpret the results.
5. Look at model fit, interpret the results.
6. Check if the model has outliers, multicollinearity, non-linear effects, and non-constant error variance. 

In [ ]:
values = values.dropna()
# Constructing a model
model = smf.ols(formula='achiv ~ cntry + stim + hedon + benev + intgndr + intage1 + gndr + age + eduy + domicil', data=values).fit()
print(model.summary())

## Results interpretation:

In [ ]:
model.pvalues[model.pvalues > 0.05]

**All coefficients are significant** except:
- Croatia
- Hungary
- Poland
- Slovakia

Listed countries will be not interpreted in the further model exploration!

In [ ]:
# Interaction effect between respondent’s age and respondent’s gender
model0 = smf.ols(formula='achiv ~ cntry + stim + hedon + benev + intgndr + intage1 + age*gndr + eduy + domicil', data=values).fit()
print('Age & Gender interaction with coef = ' + model0.summary().tables[1][-2][1].data.strip() + ' and p-value =' + model0.summary().tables[1][-2][4].data)

In [ ]:
# Visualize the interaction effect, interpret the results
sns.lmplot(x="age", y="achiv", hue="gndr", data=values, 
           truncate=0, scatter=0, line_kws={'linewidth':10},
           height=9, aspect=12/8).set(yscale="log");

Interaction effect truly exists and **gender** significantly moderates effect of **age** - as can be seen overall trend with more years of age to values achievements lower but! for **males that downtrend is elaborated with much lower slope** than for females

In [ ]:
# Look at model fit, interpret the results
model.summary().tables[0]

R-sqaured and Adjusted R-squared are equal (which seems strange to me because I await for lower adjusted R-squared with such amount of regressors)

They both equals to 29.5-4% of explained variance which is not really cool with that amount of regressors.

## Assumptions

In [ ]:
# outliers
(c, p) = model.get_influence().cooks_distance
with mpl.rc_context():
    mpl.rc("figure", figsize=(20, 15))
    plt.stem(np.arange(len(c)), c, markerfmt=",");

Almost any observation looks to be *influential* - but that highly affected by number of observations.

In [ ]:
# multicollinearity
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
X = add_constant(values[['achiv', 'age']].dropna())
pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], 
          index=X.columns)

As can be seen by VIF value for constant variable (intercept) **there is multicollinearity** for the outcome (but not for numeric regressors).

stim                                            0.2381      0.005     46.561      0.000       0.228       0.248
hedon                                           0.1876      0.005     34.642      0.000       0.177       0.198
benev                                           0.1892      0.007     27.918      0.000       0.176       0.202
intage1                                         0.0014      0.000      3.198      0.001       0.001       0.002
age                                            -0.0060      0.000    -20.502      0.000      -0.007      -0.005
eduy     

In [ ]:
### non-linear effects
with mpl.rc_context():
    mpl.rc("figure", figsize=(10, 7))
    smg.plot_ccpr(model, 'stim')

In [ ]:
with mpl.rc_context():
    mpl.rc("figure", figsize=(10, 7))
    smg.plot_ccpr(model, 'hedon')

In [ ]:
with mpl.rc_context():
    mpl.rc("figure", figsize=(10, 7))
    smg.plot_ccpr(model, 'benev')

In [ ]:
with mpl.rc_context():
    mpl.rc("figure", figsize=(10, 7))
    smg.plot_ccpr(model, 'intage1')

In [ ]:
with mpl.rc_context():
    mpl.rc("figure", figsize=(10, 7))
    smg.plot_ccpr(model, 'age')

In [ ]:
with mpl.rc_context():
    mpl.rc("figure", figsize=(10, 7))
    smg.plot_ccpr(model, 'eduy')

### No any non-linear relationships for numeric regressors are found.

In [ ]:
# non-constant error variance
print(sms.het_breuschpagan(model0.resid, values[['achiv', 'age']].dropna())[1::2])

print(sms.het_goldfeldquandt(model0.resid, values[['achiv', 'age']].dropna()))

Both tests **indicate violation of homoscedasticity.**